In [23]:
# --- Setup ---
import pandas as pd
import numpy as np
from pathlib import Path

from _init_path import *

# --- Paths ---
base = Path.cwd().parent          # assumes you're in /notebooks
raw_path = base / "data" / "raw"
processed_path = base / "data" / "processed"
processed_path.mkdir(parents=True, exist_ok=True)

# Load Data

In [24]:
# --- Load raw data ---
df_raw = pd.read_csv(raw_path / "USvideos.csv")
df_subscribers = pd.read_csv(raw_path / "USvideos_with_subscribers.csv")

# Clean Data

In [ ]:
# Clean datatypes
for col in ["views", "likes", "dislikes", "comment_count"]:
    df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

# Convert publish time strings to datetime objects
df_raw["publish_time"] = pd.to_datetime(df_raw["publish_time"], errors="coerce")

# Drop rows missing critical info
df_raw = df_raw.dropna(subset=["views", "likes", "comment_count"])

In [ ]:
# Merge secondary dataset with subscriber columns into main dataset without subscriber columns
df_merged = pd.merge(
    df_raw,
    df_subscribers[["video_id", "subscriber"]],
    on="video_id",
    how="left"
)

In [27]:
print("Subscriber data missing for:",
      round(df_merged["subscriber"].isna().mean() * 100, 1), "% of rows")

print("Unique video_ids:", df_merged["video_id"].nunique())

# --- Deduplicate (keep row with max views per video_id) ---
df_final = df_merged.loc[df_merged.groupby("video_id")["views"].idxmax()]
print("Before:", len(df_merged))
print("After deduplication:", len(df_final))

Subscriber data missing for: 44.8 % of rows
Unique video_ids: 6351
Before: 40949
After deduplication: 6351


In [35]:
# Fill missing subscriber counts using channel averages
df_final["subscriber"] = df_final.groupby("channel_title")["subscriber"].transform(lambda x: x.fillna(x.mean()))

# Drop remaining NaN subscriber values (channels with none recorded)
df_final = df_final.dropna(subset=["subscriber"])

print("Remaining rows:", len(df_final))
print("Missing subscribers:", df_final["subscriber"].isna().sum())


Remaining rows: 5905
Missing subscribers: 0


In [ ]:
# Save cleaned dataset 
out_path = processed_path / "youtube_engagement_clean.parquet"

df_final.to_parquet(out_path, index = False)

print("✅ Saved final cleaned dataset to:", out_path)
print("Rows:", len(df_final), "Columns:", len(df_final.columns))

✅ Saved final cleaned dataset to: c:\Users\johnr\OneDrive\Desktop\ML Project\YouTube_Clickability_Study\data\processed\youtube_engagement_clean.parquet
Rows: 5905 Columns: 17


In [38]:
df_check = pd.read_parquet(out_path)
print("Loaded rows:", len(df_check))
df_check.head()


Loaded rows: 5905


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,subscriber
0,-0CMnp02rNY,18.11.06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04 13:00:00+00:00,"ellen|""ellen degeneres""|""the ellen show""|""elle...",800359,9773,332,423,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2.376002e+07
1,-0NYY8cqdiQ,18.01.02,Megan Mullally Didn't Notice the Interesting P...,TheEllenShow,24,2018-01-29 14:00:39+00:00,"megan mullally|""megan""|""mullally""|""will and gr...",563746,4429,54,94,https://i.ytimg.com/vi/-0NYY8cqdiQ/default.jpg,False,False,False,Ellen and Megan Mullally have known each other...,2.376002e+07
2,-1Hm41N0dUs,18.01.05,Cast of Avengers: Infinity War Draws Their Cha...,Jimmy Kimmel Live,23,2018-04-27 07:30:02+00:00,"jimmy|""jimmy kimmel""|""jimmy kimmel live""|""late...",2058516,41248,580,1484,https://i.ytimg.com/vi/-1Hm41N0dUs/default.jpg,False,False,False,"Benedict Cumberbatch, Don Cheadle, Elizabeth O...",1.126290e+07
3,-1yT-K3c6YI,17.02.12,YOUTUBER QUIZ + TRUTH OR DARE W/ THE MERRELL T...,Molly Burke,22,2017-11-28 18:30:43+00:00,"youtube quiz|""youtuber quiz""|""truth or dare""|""...",231341,7734,212,846,https://i.ytimg.com/vi/-1yT-K3c6YI/default.jpg,False,False,False,Check out the video we did on the Merrell Twin...,2.740040e+05
4,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12 02:39:01+00:00,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...,1.477180e+05
